# KPNA2 gene analysis

This notebook can be run locally or on a remote cloud computer by clicking the badge below:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/statisticalbiotechnology/cb2030/master?filepath=nb%2Flinear%2FKPNA2.ipynb) or [![CoLab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statisticalbiotechnology/cb2030/blob/master/nb/linear/KPNA2.ipynb)


This example is taken from from [statomics](https://statomics.github.io).
Data from https://doi.org/10.1093/jnci/djj052

### Background
Histologic grade in breast cancer provides clinically important prognostic information. Researchers examined whether histologic grade was associated with gene expression profiles of breast cancers and whether such profiles could be used to improve histologic grading. In this tutorial we will assess the impact of histologic grade on expression of the KPNA2 gene that is known to be associated with poor BC prognosis. The patients, however, do not only differ in the histologic grade, but also on their lymph node status. The lymph nodes were not affected (0) or surgically removed (1). We first load our data.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from urllib.request import urlopen

try:
    gene_table = pd.read_csv('brc.txt')
except:  # recent pandas can read URL without urlopen
    url = 'https://raw.githubusercontent.com/statOmics/statisticalGenomicsCourse/master/tutorial1/gse2990BreastcancerOneGene.txt'
    fh = urlopen(url)
    gene_table = pd.read_table(fh, sep=" ")
    gene_table.to_csv('brc.txt')
gene_table.drop(columns=['Unnamed: 0'], inplace=True)


# Analysis

We first log the KPNA2 gene expression values. It is common to assume a log normal distribution of transcription values. 

In [ ]:
gene_table["log_gene"] = np.log(gene_table["gene"])
gene_table

We first plot the exression values of the KPNA2 gene for grade 1 and grade 3 cancers, and compare the ones sitting in patients where lymph nodes are or ar not surgically removed. 

In [ ]:
sns.boxplot(y="log_gene",x="grade",hue="node",data=gene_table)

Overall it seems like there is a large differnce in KPNA2 expression between grade 1 and grade 3 cancers. The size of the difference seems also depend on of lymph nodes have been removed or not. Let's test if the differences are significant. We do so by testing if the variable `log_gene` is dependent on the variable `grade`. For the test we indicate that the grade is a categorical variable, i.e. there is no linear relation between its value (1/3) and its effect. We do for instance not expect a three time as high effect for grade 3 patients as grade 1. We indicate this by by wrapping the variable with `C(.)`.

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = 'log_gene ~ C(grade)'
lm = ols(formula, gene_table).fit()
print(anova_lm(lm))

The difference un gene expression between the grades is indeed very significant.  

We expand the model to also test for differences for removed lymph nodes, as well as an interaction term between cancer grade and node removal.  The model reports three different $p$ values, one for the grade in itself, `C(grade)`, one for the node `C(node)`, and one for their interaction, `C(grade):C(node)`. 

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = 'log_gene ~ C(grade) + C(node) + C(grade):C(node)'
lm = ols(formula, gene_table).fit()
#print(lm.summary())
print(anova_lm(lm))

All three terms are significant on a p<0.05 level. So there is a difference in expression between patients with and without surgicaly removed lymph nodes, between grade 1 and grade 3 tumors, and the two previously changes are linked. Visually, this makes sence by our previous boxplot, as the mean expression values of node 0/1 differ with differnt signs for grade 1 and grade 3 tumors.  

# Test with continous independent variables
Now we continue by investigating if we can see any significant differences of KPNA2 expressions given their size.

In [ ]:
sns.lmplot(y="log_gene",x="size",hue="node",col="grade",data=gene_table)

We first test if there is a significant dependence on tumor size, first alone and subsequently also counting away the effects of the difference in grade and node status.

In [ ]:
formula = 'log_gene ~ size'
lm2 = ols(formula, gene_table).fit()
print(anova_lm(lm2))

In [ ]:
formula = 'log_gene ~ C(grade) + C(node) + size'
lm2 = ols(formula, gene_table).fit()
print(anova_lm(lm2))

In either of the tests the KPNA2 expression do not significantly depend on tumor size. 

We then investigate if there is an interaction between size and tumor grade. 

In [ ]:
formula = 'log_gene ~ C(grade) + size + size:C(grade)'
lm3 = ols(formula, gene_table).fit()
print(anova_lm(lm3))

The model reports three separate $p$ values, one for grade, one for size and one for the interaction. The test suggest that KPNA2 expression depends on an interaction between tumor size and grade. 